In [1]:

import torch
from torch.utils.data import DataLoader
from torch import nn

import numpy as np
import pandas as pd

import sys
from tqdm import tqdm


In [2]:

def CustomDataloader(train_csv: str, test_csv: str, batch_size: int = 100,
                     validation_percent: float = 0.2, shuffle: bool = False) \
                        -> tuple[DataLoader, DataLoader, DataLoader]:

    train = pd.read_csv(train_csv, header=None).to_numpy()
    test = pd.read_csv(test_csv, header=None).to_numpy()

    train_ds = torch.utils.data.TensorDataset(
        torch.from_numpy(train[:, :-1]).float(),
        torch.from_numpy(train[:, -1]).long(),
    )
    test_ds = torch.utils.data.TensorDataset(
        torch.from_numpy(test[:, :-1]).float(),
        torch.from_numpy(test[:, -1]).long(),
    )

    train_len = train.shape[0]
    val_len = int(train_len * validation_percent)
    train_len -= val_len

    train_ds, val_ds = torch.utils.data.random_split(train_ds, [train_len, val_len])

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=shuffle)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=shuffle)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=shuffle)

    return train_loader, val_loader, test_loader


In [3]:

class ConvNet(nn.Module):

    def __init__(self, num_of_class: int) -> None:

        super(ConvNet, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=3, stride=1, padding=1),
            nn.MaxPool1d(2),
            nn.Conv1d(16, 64, kernel_size=3, stride=1, padding=1),
            nn.MaxPool1d(2),
            nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.MaxPool1d(2),
        )

        self.fc = nn.Sequential(
            nn.Linear(2944, 500),
            nn.LeakyReLU(inplace=True),
            nn.Linear(500, num_of_class),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.unsqueeze(1)
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


In [4]:
batch_size = 1000
learning_rate = 3e-3
epochs = 50

#torch.manual_seed(12345)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using {} for training".format(device))

def evalute(model: torch.nn.Module, loader: DataLoader):
    model.eval()
    correct = 0
    total = len(loader.dataset)
    val_bar = tqdm(loader, file=sys.stdout)
    for x, y in val_bar:
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            logits = model(x)
            pred = logits.argmax(dim=1)
        correct += torch.eq(pred, y).sum().float().item()

    return correct / total

def main():

    train_loader, val_loader, test_loader = CustomDataloader(
        './archive/mitbih_train.csv',
        './archive/mitbih_test.csv',
        batch_size=batch_size,
        validation_percent=0.2,
        shuffle=True,
    )

    model = ConvNet(5).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = torch.nn.CrossEntropyLoss()

    best_acc, best_epoch = 0, 0
    global_step = 0

    for epoch in range(epochs):

        train_tqdm = tqdm(train_loader, file=sys.stdout)

        for step, (x, y) in enumerate(train_tqdm):

            # x.shape = [batch_size, 187], y.shape = [batch_size]
            x, y = x.to(device), y.to(device)

            model.train()
            loss = criterion(model(x), y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_tqdm.desc = "train epoch[{}/{}] loss:{:.3f}".format(epoch + 1, epochs, loss)
            global_step += 1

        if epoch % 10 == 9:
            val_acc = evalute(model, val_loader)
            print('val_acc = {}'.format(val_acc))

            if val_acc > best_acc:
                best_epoch = epoch
                best_acc = val_acc
                torch.save(model.state_dict(), 'best.pt')

    print('best acc: {:.3f}, at epoch {}.'.format(best_acc, best_epoch))

    model.load_state_dict(torch.load('best.pt'))
    print('loaded from checkpoint!'); 

    test_acc = evalute(model, test_loader)
    print('test acc = {:.3f}'.format(test_acc))

if __name__ == '__main__':
    main()


Using cpu for training
100%|██████████| 18/18 [00:02<00:00,  8.46it/s]
val_acc = 0.9717875499714449
100%|██████████| 18/18 [00:02<00:00,  8.49it/s]
val_acc = 0.9820673900628213
100%|██████████| 18/18 [00:02<00:00,  8.43it/s]
val_acc = 0.9823529411764705
100%|██████████| 18/18 [00:02<00:00,  8.49it/s]
val_acc = 0.983038263849229
100%|██████████| 18/18 [00:02<00:00,  8.24it/s]
val_acc = 0.9841233580810965
best acc: 0.9841233580810965, at epoch 49.
loaded from checkpoint!
100%|██████████| 22/22 [00:02<00:00,  8.55it/s]
test acc = 0.982733418600402
